In [8]:
# Mount gdrive folders on this colab session
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/dev/text-to-signature/archive.zip -d /content/drive/MyDrive/dev/text-to-signature

In [9]:
import numpy as np
import os
!pip install tensorflow
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [19]:
# Il dataset è composto da 1939 immagini, scala di grigi, 28x28, png, nome:nnnn_x.png
dataset_path = '/content/drive/MyDrive/dev/text-to-signature/df_resized'
conditions = []
statistics = {}

# List all files in the directory
for file_name in os.listdir(dataset_path):
    full_path = os.path.join(dataset_path, file_name)

    if os.path.isfile(full_path):  # Check if it's a file
        n = file_name.split('_')[1].split('.')[0]

        if n in statistics:
          statistics[n] += 1
        else:
          statistics[n] = 1
          conditions.append(n)

conditions.sort()
print(conditions)

statistics = dict(sorted(statistics.items()))
print(statistics)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'a': 168, 'b': 53, 'c': 50, 'd': 70, 'e': 235, 'f': 20, 'g': 48, 'h': 88, 'i': 118, 'j': 7, 'k': 32, 'l': 59, 'm': 61, 'n': 129, 'o': 136, 'p': 31, 'q': 5, 'r': 153, 's': 123, 't': 91, 'u': 83, 'v': 21, 'w': 71, 'x': 6, 'y': 79, 'z': 3}


In [4]:
def build_generator(noise_dim, condition_dim):
    noise_input = Input(shape=(noise_dim,))
    condition_input = Input(shape=(condition_dim,))

    # Concatenazione del rumore e condizione
    combined_input = Concatenate()([noise_input, condition_input])

    x = Dense(256, activation='relu')(combined_input)
    x = Dense(512, activation='relu')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(128 * 128, activation='tanh')(x)
    img_output = Reshape((128, 128, 1))(x)

    return Model([noise_input, condition_input], img_output)

In [5]:
def build_discriminator(img_shape, condition_dim):
    img_input = Input(shape=img_shape)
    condition_input = Input(shape=(condition_dim,))

    # Concatenazione immagine e condizione
    condition_expand = Dense(np.prod(img_shape))(condition_input)
    condition_expand = Reshape(img_shape)(condition_expand)
    combined_input = Concatenate()([img_input, condition_expand])

    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(combined_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)

    return Model([img_input, condition_input], x)